# Identify top biological pathways linked to blood pressure genes by the _GiGpBP_ metapath

Proposed in https://github.com/greenelab/hetmech/pull/77.

In [1]:
import collections

import pandas
import hetio.readwrite
import numpy

from hetmech.degree_weight import dwpc

In [2]:
repo_url = 'https://github.com/dhimmel/hetionet'
commit = '6d26d15e9055b33b4fd97a180fa288e4f2060b96'
names = ['hetionet-v1.0'] + [f'hetionet-v1.0-perm-{i + 1}' for i in range(5)]    
paths = ['hetnet/json/hetionet-v1.0.json.bz2'] + [
    f'hetnet/permuted/json/{name}.json.bz2' for name in names[1:]
]
hetnets = collections.OrderedDict()
for name, path in zip(names, paths):
    url = f'{repo_url}/raw/{commit}/{path}'
    hetnets[name] = hetio.readwrite.read_graph(url)

In [3]:
list(hetnets)

['hetionet-v1.0',
 'hetionet-v1.0-perm-1',
 'hetionet-v1.0-perm-2',
 'hetionet-v1.0-perm-3',
 'hetionet-v1.0-perm-4',
 'hetionet-v1.0-perm-5']

In [4]:
DWPCs = collections.OrderedDict()
for name, graph in hetnets.items():
    metapath = graph.metagraph.metapath_from_abbrev('GiGpBP')
    rows, cols, dwpc_matrix, seconds = dwpc(graph, metapath, damping=0.4)
    DWPCs[name] = dwpc_matrix
    print(f'Computing DWPC matrix for the {metapath} metapath in {name} took {seconds:.1f} seconds')

Computing DWPC matrix for the GiGpBP metapath in hetionet-v1.0 took 449.6 seconds
Computing DWPC matrix for the GiGpBP metapath in hetionet-v1.0-perm-1 took 180.7 seconds
Computing DWPC matrix for the GiGpBP metapath in hetionet-v1.0-perm-2 took 178.7 seconds
Computing DWPC matrix for the GiGpBP metapath in hetionet-v1.0-perm-3 took 178.7 seconds
Computing DWPC matrix for the GiGpBP metapath in hetionet-v1.0-perm-4 took 174.0 seconds
Computing DWPC matrix for the GiGpBP metapath in hetionet-v1.0-perm-5 took 176.4 seconds


Note that gneerating the DWPC matrices on the unpermuted network took longer. We may want to investigate the cause of this differential runtime, as it may provide a valuable insight.

In [5]:
metapath.get_unicode_str()

'Gene–interacts–Gene–participates–Biological Process'

## Read diffex

In [6]:
# Differentially expressed blood pressure genes from https://doi.org/10.1371/journal.pgen.1005035
url = 'https://doi.org/10.1371/journal.pgen.1005035.s006'
bp_df = (
    pandas.read_excel(url, skiprows=[0, 2])
    .rename(columns={
        'EntrezGeneID_FHS': 'entrez_gene_id',
    })
    .dropna(subset=['entrez_gene_id'])
    .drop_duplicates(subset=['entrez_gene_id'])
    .query("BP_sixCohort_meta_p < 0.001")
    [['entrez_gene_id', 'BP_sixCohort_meta_TE', 'BP_sixCohort_meta_p']]
)

# Entrez Genes should be ints
bp_df.entrez_gene_id = bp_df.entrez_gene_id.astype(int)

# Replace p-values that are zero
bp_df.loc[bp_df.BP_sixCohort_meta_p == 0, 'BP_sixCohort_meta_p'] = 1e-15
bp_df['weight'] = bp_df.BP_sixCohort_meta_TE * -numpy.log10(bp_df.BP_sixCohort_meta_p)
bp_df['weight_down'] = numpy.maximum(-bp_df.weight, 0)
bp_df['weight_up'] = numpy.maximum(bp_df.weight, 0)

bp_df.head(2)

,entrez_gene_id,BP_sixCohort_meta_TE,BP_sixCohort_meta_p,weight,weight_down,weight_up
0,1318,0.002282,1.000000e-15,0.034224,0.0,0.034224
1,91663,0.002578,1.000000e-15,0.038671,0.0,0.038671


In [7]:
pandas.Series(numpy.sign(bp_df.weight)).value_counts()

 1.0    68
-1.0    65
Name: weight, dtype: int64

In [8]:
gene_df = (
    pandas.DataFrame({
        'entrez_gene_id': rows,
        'gene_symbol': [graph.get_node((metapath.source().identifier, x)).name for x in rows],
    })
    .merge(bp_df, how='left')
    [['entrez_gene_id', 'gene_symbol', 'weight', 'weight_down', 'weight_up']]
    .fillna(0)
)

gene_df.head(2)

,entrez_gene_id,gene_symbol,weight,weight_down,weight_up
0,1,A1BG,0.0,0.0,0.0
1,2,A2M,0.0,0.0,0.0


## Compute target node scores

In [9]:
target_df = pandas.DataFrame({
    'metapath': str(metapath),
    'target_id': cols,
    'target_name': [graph.get_node((metapath.target().identifier, x)).name for x in cols],
}).set_index(['metapath', 'target_id', 'target_name'])

for name, array in DWPCs.items():
    target_df[name] = gene_df.weight_up @ array

# Scaling as per https://think-lab.github.io/d/193/#4
dwpc_scaler = target_df['hetionet-v1.0'].mean()
target_df = numpy.arcsinh(target_df / dwpc_scaler)

perm_df = target_df.iloc[:, 1:]
target_df['z-score'] = (target_df.iloc[:, 0] - perm_df.mean(axis='columns')) / perm_df.std(axis='columns')

(
    target_df
    # Remove targets without sufficient nonzero DWPCs
    [(perm_df > 0).sum(axis='columns') >= 3]
    .sort_values('z-score', ascending=False)
    .head(20)
)

hetionet-v1.0  \
metapath target_id  target_name                                                         
GiGpBP   GO:0051208 sequestering of calcium ion                              1.293332   
         GO:0072236 metanephric loop of Henle development                    1.240933   
         GO:0061299 retina vasculature morphogenesis in camera-type...       0.835832   
         GO:0070426 positive regulation of nucleotide-binding oligo...       1.759714   
         GO:0070318 positive regulation of G0 to G1 transition               1.376872   
         GO:0048936 peripheral nervous system neuron axonogenesis            1.826185   
         GO:0030316 osteoclast differentiation                               1.415508   
         GO:0032464 positive regulation of protein homooligomerization       2.260878   
         GO:0090400 stress-induced premature senescence                      1.383377   
         GO:1901099 negative regulation of signal transduction in a...       1.658419   
         GO:0030050 vesicle transport along actin filament                   1.457428   
         GO:1903265 positive regulation of tumor necrosis factor-me...       1.821209   
         GO:0071287 cellular response to manganese ion                       1.607277   
         GO:0090435 protein localization to nuclear envelope                 0.634820   
         GO:0036015 response to interleukin-3                                1.332852   
         GO:0033693 neurofilament bundle assembly                            2.810967   
         GO:0052040 modulation by symbiont of host programmed cell ...       1.382360   
         GO:2000644 regulation of receptor catabolic process                 0.808170   
         GO:0002758 innate immune response-activating signal transd...       2.778018   
         GO:0097084 vascular smooth muscle cell development                  2.300940   

                                                                        hetionet-v1.0-perm-1  \
metapath target_id  target_name                                                                
GiGpBP   GO:0051208 sequestering of calcium ion                                     0.000000   
         GO:0072236 metanephric loop of Henle development                           0.048121   
         GO:0061299 retina vasculature morphogenesis in camera-type...              0.075949   
         GO:0070426 positive regulation of nucleotide-binding oligo...              0.063897   
         GO:0070318 positive regulation of G0 to G1 transition                      0.102426   
         GO:0048936 peripheral nervous system neuron axonogenesis                   0.121804   
         GO:0030316 osteoclast differentiation                                      0.593321   
         GO:0032464 positive regulation of protein homooligomerization              0.000000   
         GO:0090400 stress-induced premature senescence                             0.000000   
         GO:1901099 negative regulation of signal transduction in a...              0.286770   
         GO:0030050 vesicle transport along actin filament                          0.000000   
         GO:1903265 positive regulation of tumor necrosis factor-me...              0.102778   
         GO:0071287 cellular response to manganese ion                              0.216493   
         GO:0090435 protein localization to nuclear envelope                        0.048058   
         GO:0036015 response to interleukin-3                                       0.217064   
         GO:0033693 neurofilament bundle assembly                                   0.284976   
         GO:0052040 modulation by symbiont of host programmed cell ...              0.017887   
         GO:2000644 regulation of receptor catabolic process                        0.094987   
         GO:0002758 innate immune response-activating signal transd...              1.789522   
         GO:0097084 vascular smooth muscle cell development                         0.000000   

